#Group 3
Authors:

Barrion, Ryan Onil

Geronimo, Michael Robert

Roledo, Jan Carlo

Solis, Frances Danielle

---
# C++ Version of Dot Product

In [ ]:
%%writefile c_dotp.c
#include <bits/stdc++.h>
#include <ctime>
using namespace std;


//C++ Dot Product Function
void dot_product (int n, double *h_out, float* h_in1, float* h_in2){
  for (int i = 0; i < n; i++)
   *h_out += h_in1[i] * h_in2[i];
}

int main() {
    const unsigned int ARRAY_SIZE = 1<<24;
    const unsigned ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    const unsigned int run = 30;

    float *in1, *in2;
    double out = 0;
    in1 = (float*)malloc(ARRAY_BYTES);
    in2 = (float*)malloc(ARRAY_BYTES);

    clock_t start, end;

    //init data
    for(int i = 0; i <ARRAY_SIZE;i++){
        in1[i]=float(i);
        in2[i]=float(i);
    }
    
    dot_product(ARRAY_SIZE, &out, in1, in2);


    double time_taken;
    for(int i = 0; i<run; i++)
      out = 0;
      start = clock();
      dot_product(ARRAY_SIZE, &out, in1, in2);
      end = clock();
      time_taken += (double)end-start;
    
    time_taken = ((time_taken)*1e6/CLOCKS_PER_SEC)/run;
    printf("C++ function took an average of %lf microseconds for array size %d.\n", time_taken, ARRAY_SIZE);
  
    //Check for errors
    double sum = 0;
    for(int i = 0; i < ARRAY_SIZE; i++){
      sum += in1[i] * in2[i];
    }

    cout << "Output: " << out << endl;
    cout << "Expected: " << sum << endl;

    double p_error = abs((out - sum) / sum) * 100.0;

    cout << "Percent Error: " << p_error << endl;
   
   //free memory
   free(in1);
   free(in2);

   return 0;

}

Writing c_dotp.c


In [ ]:
%%shell
g++ c_dotp.c -o c_dotp

In [ ]:
%%shell
./c_dotp

C++ function took an average of 1766.433333 microseconds for array size 16777216.
Output: 1.57412e+21
Expected: 1.57412e+21
Percent Error: 0


___
#2. CUDA Without Cooperative Group Dot Product

In [ ]:
%%writefile cuda_uncooperative_threads.cu

#include <stdio.h>
#include <stdlib.h>

__global__
void dotProduct(float *x,float *y, double *dotp,long unsigned int ARRAY_SIZE){

  extern __shared__ double mul_result[];
  mul_result[threadIdx.x]  = 0.0;


  int index = blockIdx.x * gridDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;


  for (int i = blockIdx.x * gridDim.x + threadIdx.x; 
        i < ARRAY_SIZE;
        i += blockDim.x * gridDim.x){
      mul_result[threadIdx.x] += (double)y[i] * x[i];
  }
  __syncthreads();

  if(threadIdx.x == 0){
    double sum = 0.0;

    for(int i = 0 ; i < blockDim.x; ++i){
      sum = sum + mul_result[i];
    }
    atomicAdd(dotp, sum);
  }

}
int main(void){

  //number of runs
  int runs = 30;

  // Declare Number of Elements
  const unsigned long int ARRAY_SIZE = 1<<24;
  const unsigned long int ARRAY_BYTES = ARRAY_SIZE * sizeof(ARRAY_SIZE); 

  // Declare Dimensions
  const int nBlocks = 1024;
  const int nThreads = nBlocks;

  int device = -1;
  cudaGetDevice(&device);

  float *x, *y;
  double *dotp;

  // Allocate resources for variables x and y;
  cudaMallocManaged(&x,ARRAY_BYTES);
  cudaMallocManaged(&y, ARRAY_BYTES);

  // Allocate only a single space to store the result of sum
  cudaMallocManaged(&dotp, sizeof(double));
  *dotp = 0.0;
  
  //Prefetch Part 1
  cudaMemAdvise(x,ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(y,ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);

  cudaMemPrefetchAsync(dotp,sizeof(double), device, NULL);

  // Initialize Variables
  for (int i = 0; i < ARRAY_SIZE; ++i){
    x[i] = (float)((i + 1) % 50);
    y[i]  = x[i] + 2.0;
  }


  // Prefetch Part 2
  cudaMemAdvise(x, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);
  cudaMemAdvise(y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);

  cudaMemAdvise(x, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemAdvise(y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

  cudaMemPrefetchAsync(x,ARRAY_BYTES, device, NULL);
  cudaMemPrefetchAsync(y,ARRAY_BYTES, device, NULL);

  // Initial launch to prevent additional time for cache miss

  //Launch Kernel
  for (int i=0; i<runs; i++){
    cudaMemPrefetchAsync(dotp, sizeof(double), cudaCpuDeviceId, NULL);
    *dotp = 0;
    cudaMemPrefetchAsync(dotp, sizeof(double), device, NULL);

    dotProduct<<<nBlocks,nThreads, sizeof(double) * nThreads>>>(x,y,dotp,ARRAY_SIZE);
    cudaDeviceSynchronize();
  }
  
  // Prefetch Result
  cudaMemPrefetchAsync(dotp, sizeof(double), cudaCpuDeviceId, NULL);

  // Check for Error by measuring percent error
  double expected = 0;

  for(int i = 0; i < ARRAY_SIZE; ++i){
    expected = expected + (double)(x[i] * y[i]);
  }

  float pError = abs((dotp[0]-expected)/expected) * 100;

  printf("\n\nNumber of Elements: %lu\n",ARRAY_SIZE);
  printf("Number of Blocks: %d\n",nBlocks);
  printf("Number of Threads: %d\n",nThreads);

  printf("\n\nExpected:%f\nComputed:%f\n\n", dotp[0], expected );
  printf("Percent Error: %f\n\n",pError );

  cudaFree(x);
  cudaFree(y);
  cudaFree(dotp);

  return 0;
}

Writing cuda_uncooperative_threads.cu


In [ ]:
%%shell

nvcc cuda_uncooperative_threads.cu -o cuda_uncooperative_threads -arch=sm_60 

nvprof ./cuda_uncooperative_threads


cuda_uncooperative_threads.cu(12): warning #177-D: variable "index" was declared but never referenced

cuda_uncooperative_threads.cu(13): warning #177-D: variable "stride" was declared but never referenced

==15994== NVPROF is profiling process 15994, command: ./cuda_uncooperative_threads


Number of Elements: 16777216
Number of Blocks: 1024
Number of Threads: 1024


Expected:14386450768.000000
Computed:14386450768.000000

Percent Error: 0.000000

==15994== Profiling application: ./cuda_uncooperative_threads
==15994== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  104.84ms        30  3.4946ms  3.4928ms  3.4960ms  dotProduct(float*, float*, double*, unsigned long)
      API calls:   73.36%  376.92ms         3  125.64ms  22.604us  376.84ms  cudaMallocManaged
                   21.02%  108.01ms        30  3.6002ms  3.4966ms  6.4880ms  cudaDeviceSynchronize
                    2.62%  13.482ms         3  4.4939m

---
---
#3. CUDA Cooperative Group Dot Product
##3.1 Replacing __syncthreads with thread_block.sync()

In [ ]:
%%writefile cuda_group.cu

#include <stdio.h>
#include <math.h>
#include <cooperative_groups.h>

using namespace cooperative_groups;
namespace cg = cooperative_groups;

#define BLOCK_SIZE 1024

//device "sub-function" to generate a partial sum of the dot products of 4 consecutive elements
__device__ double thread_dot(float *input1, float *input2, int n) 
{
    double sum = 0;

    //grid-stride loop 
    for(int i = blockIdx.x * blockDim.x + threadIdx.x;
        i < n / 4; //only 1/4 because float4 takes 4 elements
        i += blockDim.x * gridDim.x)
    {
        //fetch 4 inputs at once, into a 4-element vector
        float4 in1 = ((float4*)input1)[i];
        float4 in2 = ((float4*)input2)[i];

        //perform partial dot product of 4 elements at a time
        sum += (in1.x*in2.x) + (in1.y*in2.y) + (in1.z*in2.z) + (in1.w*in2.w);
    }
    return sum;
}



//device "sub-function" to fill shared memory and add elements in parallel
__device__
double reduction_dotprod(thread_block g, double *temp, double val){

  //get thread index
  int lane = g.thread_rank();

  //binary reduction loop
  for (int i = g.size() / 2; i > 0; i /= 2){        
    temp[lane] = val;
    g.sync(); //wait for threads to store
    if (lane < i) val += temp[lane + i];
    g.sync(); //wait for threads to load
  }

  return val;
}


//main kernel function
//initializes thread blocks and combines results of 
__global__
void dotprod_group(int n, float *in_x, float *in_y, double *out){

  double init_sum = thread_dot(in_x, in_y, n);

  //create shared memory space
  extern __shared__ double temp[];

  //initialize thread block for binary-reduction sum
  auto g = this_thread_block();

  //perform reduction sum
  double block_sum = reduction_dotprod(g, temp, init_sum);

  //the first thread (thread 0) will have the final result
  if (threadIdx.x == 0){
    atomicAdd(out, block_sum);
  }

}


int main() {

  //number of runs
  int runs = 30;

  const unsigned int ARRAY_SIZE = 1<<24;

  const unsigned int ARRAY_BYTES = ARRAY_SIZE * sizeof(int);

  //size of shared bytes in sum_group() kernel
  const unsigned int SHARED_BYTES = BLOCK_SIZE * sizeof(double);

  const int NUM_BLOCKS = BLOCK_SIZE;
  printf("numElem = %d\tnumThreads = %d\tnumBlocks = %d\n", ARRAY_SIZE, BLOCK_SIZE, NUM_BLOCKS);

  //declare ARRAY_BYTES or allocate in GPU
  float *in_x, *in_y;
  double *out;
  cudaMallocManaged(&in_x, ARRAY_BYTES);
  cudaMallocManaged(&in_y, ARRAY_BYTES);
  cudaMallocManaged(&out, sizeof(double));

  //prefetch input data to CPU
  cudaMemPrefetchAsync(in_x, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(in_y, ARRAY_BYTES, cudaCpuDeviceId, NULL);

  //init data
  for(int i = 0; i < ARRAY_SIZE; i++)
    in_x[i] = in_y[i] = 0.000001f * i;

  int device = -1;
  cudaGetDevice(&device); // get device number of GPU

  cudaMemPrefetchAsync(out, sizeof(double), device, NULL);

  //Prefetch and Memory Advise input data to device
  cudaMemAdvise(in_x, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);
  cudaMemPrefetchAsync(in_x, ARRAY_BYTES, device, NULL);

  cudaMemAdvise(in_y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);
  cudaMemPrefetchAsync(in_y, ARRAY_BYTES, device, NULL);

  for(int i=0; i<runs;i++){
    cudaMemPrefetchAsync(out, sizeof(double), cudaCpuDeviceId, NULL);
    *out = 0;
    cudaMemPrefetchAsync(out, sizeof(double), device, NULL);

    //perform dot product with Cooperative Groups (thread_block)
    dotprod_group<<<NUM_BLOCKS, BLOCK_SIZE, SHARED_BYTES>>> (ARRAY_SIZE, in_x, in_y, out); 
    
    cudaDeviceSynchronize();
  }


  //prefetch input data to CPU
  cudaMemAdvise(in_x, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemPrefetchAsync(in_x, ARRAY_BYTES, cudaCpuDeviceId, NULL);

  cudaMemAdvise(in_y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemPrefetchAsync(in_y, ARRAY_BYTES, cudaCpuDeviceId, NULL);

  cudaMemAdvise(out, sizeof(double), cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemPrefetchAsync(out, sizeof(double), cudaCpuDeviceId, NULL);

  //check for errors
  double sum = 0;
  for(int i = 0; i < ARRAY_SIZE/2; i++) {
    sum += (in_x[i] * in_y[i] + in_x[i+ARRAY_SIZE/2] * in_y[i+ARRAY_SIZE/2]);
  }

  double sum_seq = 0;
  for (int i = 0; i < ARRAY_SIZE; i++){
    sum_seq += in_x[i] * in_y[i];
  }
  printf("Sequential Sum: %lf\t", sum_seq);
  printf("'Binary Split' Sum: %lf\t Coop-Group Reduction Sum: %lf\n",sum, *out);

  double diff = abs((*out - sum) / sum) * 100.0;

  printf("Difference with split sum: %.5f%%\n", diff);

  //free memory in GPU
  cudaFree(in_x);
  cudaFree(in_y);
  cudaFree(out);

  return 0;
}

Writing cuda_group.cu


In [ ]:
%%shell
nvcc cuda_group.cu -o cuda_group -arch=sm_60

In [ ]:
%%shell
nvprof ./cuda_group

numElem = 16777216	numThreads = 1024	numBlocks = 1024
==20207== NVPROF is profiling process 20207, command: ./cuda_group
Sequential Sum: 1574122012.273521	'Binary Split' Sum: 1574122012.255829	 Coop-Group Reduction Sum: 1574122010.090982
Difference with split sum: 0.00000%
==20207== Profiling application: ./cuda_group
==20207== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  19.639ms        30  654.63us  646.48us  661.84us  dotprod_group(int, float*, float*, double*)
      API calls:   76.13%  266.51ms         3  88.837ms  19.577us  266.44ms  cudaMallocManaged
                   12.18%  42.622ms        68  626.80us  13.267us  17.268ms  cudaMemPrefetchAsync
                    5.68%  19.877ms        30  662.58us  654.72us  722.33us  cudaDeviceSynchronize
                    3.92%  13.710ms         3  4.5701ms  126.81us  7.6223ms  cudaFree
                    1.95%  6.8399ms         5  1.3680ms  2.4850us  3.48

---
## 3.2 Subgrouping

By creating static sized subgroups out of the current thread blocks, we can perform some compiler optimization tricks to improve performance.

Here, we make 32-thread subgroups from the original thread group, and unroll the loop within the reduction sum device function.

In [ ]:
%%writefile cuda_group_32.cu

#include <stdio.h>
#include <math.h>
#include <cooperative_groups.h>

using namespace cooperative_groups;
namespace cg = cooperative_groups;

#define BLOCK_SIZE 1024


//device "sub-function" to generate a partial sum of the dot products of 4 consecutive elements
__device__ double thread_dot(float *input1, float *input2, int n) 
{
    double sum = 0;

    //grid-stride loop 
    for(int i = blockIdx.x * blockDim.x + threadIdx.x;
        i < n / 4; //only 1/4 because float4 takes 4 elements
        i += blockDim.x * gridDim.x)
    {
        //fetch 4 inputs at once, into a 4-element vector
        float4 in1 = ((float4*)input1)[i];
        float4 in2 = ((float4*)input2)[i];

        //perform partial dot product of 4 elements at a time
        sum += (in1.x*in2.x) + (in1.y*in2.y) + (in1.z*in2.z) + (in1.w*in2.w);
    }
    return sum;
}



//device "sub-function" to fill shared memory and add partial sums in parallel
template <typename group_t>
__device__
double reduction_dotprod(group_t g, double *temp, double val){

  //get thread index
  int lane = g.thread_rank();

  //binary reduction loop
  //unroll is possible since group size is static and known at compile-time
  #pragma unroll
  for (int i = g.size() / 2; i > 0; i /= 2){        
    temp[lane] = val;
    g.sync(); //wait for threads to store
    if (lane < i) val += temp[lane + i];
    g.sync(); //wait for threads to load
  }

  return val;
}

__global__
void dotprod_group_32(int n, float *in_x, float *in_y, double *out){

  //get partial sum `init_sum`
  double init_sum = thread_dot(in_x, in_y, n);

  //create shared memory space
  extern __shared__ double temp[];

  //initialize thread block for binary-reduction sum
  auto g = this_thread_block();

  //create 32-thread tiles and get pointer to matching temp value
  int tileIdx = g.thread_rank() / 32;
  double *t = &temp[32*tileIdx];

  //create a static sized group
  thread_block_tile<32> tile32 = tiled_partition<32>(this_thread_block());

  //perform reduction sum
  double tile_sum = reduction_dotprod<thread_block_tile<32>>(tile32, t, init_sum);

  //the first thread (thread 0) will have the final result
  if (tile32.thread_rank() == 0){
    atomicAdd(out, tile_sum);
  }

}



int main() {

  int runs = 30;

  const unsigned int ARRAY_SIZE = 1<<24;

  const unsigned int ARRAY_BYTES = ARRAY_SIZE * sizeof(int);

  //size of shared bytes in sum_group() kernel
  const unsigned int SHARED_BYTES = BLOCK_SIZE * sizeof(double);

  //matching block size and number of blocks greatly improves performance
  const int NUM_BLOCKS = BLOCK_SIZE;
  printf("numElem = %d\tnumThreads = %d\tnumBlocks = %d\n", ARRAY_SIZE, BLOCK_SIZE, NUM_BLOCKS);

  //declare ARRAY_BYTES or allocate in GPU
  float *in_x, *in_y;
  double *out;
  cudaMallocManaged(&in_x, ARRAY_BYTES);
  cudaMallocManaged(&in_y, ARRAY_BYTES);
  cudaMallocManaged(&out, sizeof(double));

  //prefetch input data to CPU
  cudaMemPrefetchAsync(in_x, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(in_y, ARRAY_BYTES, cudaCpuDeviceId, NULL);

  //init data
  for(int i = 0; i < ARRAY_SIZE; i++)
    in_x[i] = in_y[i] = 0.000001f * i;

  int device = -1;
  cudaGetDevice(&device); // get device number of GPU

  cudaMemPrefetchAsync(out, ARRAY_BYTES, device, NULL);

  //Prefetch and Memory Advise input data to device
  cudaMemAdvise(in_x, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);
  cudaMemPrefetchAsync(in_x, ARRAY_BYTES, device, NULL);

  cudaMemAdvise(in_y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);
  cudaMemPrefetchAsync(in_y, ARRAY_BYTES, device, NULL);

  cudaMemPrefetchAsync(out, sizeof(double), device, NULL);


  //perform dot product with Cooperative Groups (thread_block)
  for(int i=0; i<runs; i++){
    cudaMemPrefetchAsync(out, sizeof(double), cudaCpuDeviceId, NULL);
    *out= 0;
    cudaMemPrefetchAsync(out, sizeof(double), device, NULL);

    dotprod_group_32<<<NUM_BLOCKS, BLOCK_SIZE, SHARED_BYTES>>> (ARRAY_SIZE, in_x, in_y, out);

    cudaDeviceSynchronize();
  }


  //prefetch input data to CPU
  cudaMemAdvise(in_x, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemPrefetchAsync(in_x, ARRAY_BYTES, cudaCpuDeviceId, NULL);

  cudaMemAdvise(in_y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemPrefetchAsync(in_y, ARRAY_BYTES, cudaCpuDeviceId, NULL);

  cudaMemAdvise(out, sizeof(float), cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemPrefetchAsync(out, sizeof(float), cudaCpuDeviceId, NULL);

  //check for errors
  double sum = 0;
  for(int i = 0; i < ARRAY_SIZE/2; i++) {
    sum += (in_x[i] * in_y[i] + in_x[i+ARRAY_SIZE/2] * in_y[i+ARRAY_SIZE/2]);
  }

  double sum_seq = 0;
  for (int i = 0; i < ARRAY_SIZE; i++){
    sum_seq += in_x[i] * in_y[i];
  }
  printf("Sequential Sum: %lf\t", sum_seq);
  printf("'Binary Split' Sum: %lf\t Coop-Group Reduction Sum: %lf\n",sum, *out);

  double diff = abs((*out - sum) / sum) * 100.0;

  printf("Difference with split sum: %.5f%%\n", diff);

  //free memory in GPU
  cudaFree(in_x);
  cudaFree(in_y);
  cudaFree(out);

  return 0;
}

Writing cuda_group_32.cu


In [ ]:
%%shell
nvcc cuda_group_32.cu -o cuda_group_32 -arch=sm_60

In [ ]:
%%shell
nvprof ./cuda_group_32

numElem = 16777216	numThreads = 1024	numBlocks = 1024
==21150== NVPROF is profiling process 21150, command: ./cuda_group_32
Sequential Sum: 1574122012.273521	'Binary Split' Sum: 1574122012.255829	 Coop-Group Reduction Sum: 1574122010.090982
Difference with split sum: 0.00000%
==21150== Profiling application: ./cuda_group_32
==21150== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  15.168ms        30  505.59us  504.15us  506.90us  dotprod_group_32(int, float*, float*, double*)
      API calls:   77.12%  273.58ms         3  91.192ms  18.264us  273.52ms  cudaMallocManaged
                   13.22%  46.898ms        69  679.68us  7.0680us  18.578ms  cudaMemPrefetchAsync
                    5.10%  18.081ms        30  602.72us  507.70us  3.2729ms  cudaDeviceSynchronize
                    2.58%  9.1600ms         3  3.0533ms  109.70us  4.6055ms  cudaFree
                    1.84%  6.5100ms         5  1.3020ms  1.435

Cooperative Group Reference: https://developer.nvidia.com/blog/cooperative-groups/

Equal Thread count and Block count results in faster performance (correlational): http://selkie.macalester.edu/csinparallel/modules/CUDAArchitecture/build/html/2-Findings/Findings.html

